In [27]:
import os
import numpy as np
import pandas as pd
import time as time
from PIL import Image

In [28]:
def calculateHighest(pixel_data, width, height):
    """
    [0]Red; [1]Blue; [2]Green; [3]Transparency
    :param pixel_data:
    :param width:
    :param height:
    :return: maximum value of transparency from A band, for this case this should be the 100% of intensity of touch
    """
    total = []
    input_pixel = pixel_data
    for i in range(int(width)):  # for every col:
        for j in range(int(height)):  # for every row:
            alpha_value = input_pixel[i, j][3] / 255  # read data from the A band
            total.append(alpha_value)
    maximum = max(total)
    return maximum

In [29]:
def intensityNormalization(pixel_data, maximum):
    """

    :param pixel_data:
    :param maximum:
    :return:
    """
    input_pixel = pixel_data
    alpha_value = input_pixel[3] / 255
    new_r = int(input_pixel[0])
    new_b = int(input_pixel[1])
    new_g = int(input_pixel[2])
    new_a = int((alpha_value * 100 / maximum))
    new_pixel = (new_r, new_b, new_g, new_a)
    return new_pixel


In [30]:
dir1 = 'source/data_pilot/Men Preferred Love/'
dir2 = 'source/data_pilot/Men Preferred Sex/'
dir3 = 'source/data_pilot/Men Received Love/'
dir4 = 'source/data_pilot/Men Received Sex/'
dir5 = 'source/data_pilot/Women Preferred Love/'
dir6 = 'source/data_pilot/Women Preferred Sex/'
dir7 = 'source/data_pilot/Women Received Love/'
dir8 = 'source/data_pilot/Women Received Sex/'

all_dir = [dir1, dir2, dir3, dir4,
           dir5, dir6, dir7, dir8]

In [32]:
w = 800
h = 300

new_r = 0
new_b = 0
new_g = 0
new_a = 0

dbug = 1

for target in all_dir:
    print('Working on "', target.split('/')[2], '"')

    for r, d, f in os.walk(target):

        new_image = Image.new(mode="RGBA", size=(800, 300))
        merge_pixels = new_image.load()

        for file in f:
            if file.endswith('.png'):
                print(file)
                dbug += 1
                im = Image.open(os.path.join(r, file))
                im_pixels = im.load()
                out_pixels = im.load()
                intensity = calculateHighest(im_pixels, w, h)
                for i in range(w):
                    for j in range(h):
                        out_pixels[i, j] = intensityNormalization(im_pixels[i, j], intensity)

            # print(out_pixels)
            for i in range(w):
                for j in range(h):
                    if out_pixels[i, j][0] == 255:
                        new_r = merge_pixels[i, j][0] + 1
                        new_b = 0
                        new_g = 0
                        new_a = merge_pixels[i, j][3] + out_pixels[i, j][3]
                        new_pixel = (new_r, new_b, new_g, new_a)
                        merge_pixels[i, j] = new_pixel

        new_image.save('output/merge/' + target.split('/')[2] + ".png", "png")
        correct_color = Image.open('output/merge/' + target.split('/')[2] + ".png")
        img_array = np.array(correct_color)

        r = img_array[:, :, 0]

        mask_high = r > 1
        mask_low = r <= 1

        img_array[mask_high, 0] = 255

        img_array[mask_low, 0] = 0
        img_array[mask_low, 3] = 0

        new_img = Image.fromarray(img_array)
        new_img.save('output/merge/' + target.split('/')[2] + ".png", "png")



Working on " Men Preferred Love "
R_1YtHcvSDG01Iff3_signature.png
R_21d2uiIEZ15j015_signature.png
R_2j36Zt1QlxTgIcV_signature.png
R_2STmRYONoqJZ8WZ_signature.png
R_2SvGizciSJ8Jupo_signature.png
R_5poWufHV0ePdX0t_signature.png
R_7dmQJTCYooRF0sO_signature.png
R_8byMPmtQ3SFmMkp_signature.png
R_8MUaSTQ3mhIgKUF_signature.png
R_8QE64CXx55mPr9S_signature.png
R_8sZl23pyLHePbI2_signature.png
Working on " Men Preferred Sex "
R_1YtHcvSDG01Iff3_signature.png
R_21d2uiIEZ15j015_signature.png
R_2j36Zt1QlxTgIcV_signature.png
R_2STmRYONoqJZ8WZ_signature.png
R_2SvGizciSJ8Jupo_signature.png
R_5poWufHV0ePdX0t_signature.png
R_7dmQJTCYooRF0sO_signature.png
R_8byMPmtQ3SFmMkp_signature.png
R_8MUaSTQ3mhIgKUF_signature.png
R_8QE64CXx55mPr9S_signature.png
R_8sZl23pyLHePbI2_signature.png
Working on " Men Received Love "
R_1YtHcvSDG01Iff3_signature.png
R_21d2uiIEZ15j015_signature.png
R_2j36Zt1QlxTgIcV_signature.png
R_2STmRYONoqJZ8WZ_signature.png
R_2SvGizciSJ8Jupo_signature.png
R_5poWufHV0ePdX0t_signature.png
R_7d